In [ ]:
# google drive connect
Copied_path = '/content/drive/MyDrive/Colab Notebooks/MyPaper/GlocalSTN' # Paste target directory here
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir(Copied_path)

Mounted at /content/drive


In [ ]:
# m8_shop

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m8_Shop.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
# DST Network/ilayer.py

from keras import backend as K
from keras.engine.topology import Layer
# from keras.layers import Dense
import numpy as np


class iLayer(Layer):
    def __init__(self, **kwargs):
        # self.output_dim = output_dim
        super(iLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        initial_weight_value = np.random.random(input_shape[1:])
        self.W = K.variable(initial_weight_value)
        #self.trainable_weights = [self.W]
        self.trainable_weight = [self.W]

    def call(self, x, mask=None):
        return x * self.W

    def get_output_shape_for(self, input_shape):
        return input_shape

In [ ]:
# "metrics.py"

# import numpy as np
from keras import backend as K

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def root_mean_square_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5


# aliases
mse = MSE = mean_squared_error
# rmse = RMSE = root_mean_square_error


def masked_mean_squared_error(y_true, y_pred):
    idx = (y_true > 1e-6).nonzero()
    return K.mean(K.square(y_pred[idx] - y_true[idx]))

def masked_rmse(y_true, y_pred):
    return masked_mean_squared_error(y_true, y_pred) ** 0.5


def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))

def mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


threshold=0.05

def mean_absolute_percentage_error(y_true, y_pred):
    return K.mean( K.abs(y_pred-y_true) / K.maximum(K.cast(threshold,'float32'),y_true+1.0) )

def mape(y_true, y_pred):
    return K.mean( K.abs(y_pred-y_true) / K.maximum(K.cast(threshold,'float32'),y_true+1.0) )



In [ ]:
# "DeepSTN_net.py"

from keras import backend as K
K.set_image_data_format('channels_first')
import numpy as np

from keras.optimizers import Adam
from keras.layers import Input,Activation,Dropout,BatchNormalization,AveragePooling2D,ZeroPadding2D,Multiply
from keras.layers import Lambda,Reshape,Concatenate,Add,Permute,TimeDistributed
from keras.models import Model
from keras.layers.convolutional import Conv2D
#import PPT3_network.metrics as metrics

# Relu-BN-Conv2D 3x3
def conv_unit0(Fin,Fout,drop,H,W):
    unit_input=Input(shape=(Fin,H,W))

    unit_conv=Activation('relu')(unit_input)
    unit_conv=BatchNormalization()(unit_conv)
    unit_conv=Dropout(drop)(unit_conv)
    unit_output=Conv2D(filters=Fout,kernel_size=(3,3),padding="same")(unit_conv)
    unit_model=Model(inputs=unit_input,outputs=unit_output)
    #print('kernel=(3,3)')
    return unit_model

# Relu-BN-Conv2D 1x1
def conv_unit1(Fin,Fout,drop,H,W):
    unit_input=Input(shape=(Fin,H,W))

    unit_conv=Activation('relu')(unit_input)
    unit_conv=BatchNormalization()(unit_conv)
    unit_conv=Dropout(drop)(unit_conv)
    unit_output=Conv2D(filters=Fout,kernel_size=(1,1),padding="same")(unit_conv)
    unit_model=Model(inputs=unit_input,outputs=unit_output)
    #print('kernel=(1,1)')
    return unit_model

# efficient version of Res_plus
def Res_plus_E(name,F,Fplus,rate,drop,H,W):
    cl_input=Input(shape=(F,H,W))

    #normal channels
    cl_conv1A=conv_unit0(F,F-Fplus,drop,H,W)(cl_input)

    #separated channels
    if rate == 1:
        cl_conv1B=cl_input
    if rate !=1:
        cl_conv1B=AveragePooling2D(pool_size=(rate,rate),strides=(rate,rate),padding="valid")(cl_input)

    HR,WR=int(np.floor(H/rate)),int(np.floor(W/rate))
    cl_conv1B=Activation('relu')(cl_conv1B)
    cl_conv1B=BatchNormalization()(cl_conv1B)

    cl_conv1B=Conv2D(filters=Fplus,kernel_size=(1,1),use_bias=False,padding="same")(cl_conv1B)
    cl_conv1B=Reshape((Fplus,1,HR,WR),input_shape=(Fplus,HR,WR))(cl_conv1B)
    attention=Conv2D(filters=H*W,kernel_size=(HR,WR),use_bias=False,W_constraint=nonneg(),padding="valid")
    cl_conv1B=TimeDistributed(attention)(cl_conv1B)
    cl_conv1B=Reshape((Fplus,H,W),input_shape=(Fplus,H*W,1,1))(cl_conv1B)

    #merge
    cl_conv1=Concatenate(axis=1)([cl_conv1A,cl_conv1B])

    cl_conv2=conv_unit0(F,F,drop,H,W)(cl_conv1)

    cl_out=Add()([cl_input,cl_conv2])

    cl_model=Model(inputs=cl_input,outputs=cl_out,name=name)

    return cl_model

# new resdual block
def Res_plus(name,F,Fplus,rate,drop,H,W):
    cl_input=Input(shape=(F,H,W))

    cl_conv1A=conv_unit0(F,F-Fplus,drop,H,W)(cl_input)

    if rate == 1:
        cl_conv1B=cl_input
    if rate !=1:
        cl_conv1B=AveragePooling2D(pool_size=(rate,rate),strides=(rate,rate),padding="valid")(cl_input)

    cl_conv1B=Activation('relu')(cl_conv1B)
    cl_conv1B=BatchNormalization()(cl_conv1B)

    plus_conv=Conv2D(filters=Fplus*H*W,kernel_size=(int(np.floor(H/rate)),int(np.floor(W/rate))),padding="valid")

    cl_conv1B=plus_conv(cl_conv1B)

    cl_conv1B=Reshape((Fplus,H,W))(cl_conv1B)

    cl_conv1=Concatenate(axis=1)([cl_conv1A,cl_conv1B])

    cl_conv2=conv_unit0(F,F,drop,H,W)(cl_conv1)

    cl_out=Add()([cl_input,cl_conv2])

    cl_model=Model(inputs=cl_input,outputs=cl_out,name=name)

    return cl_model

# normal residual block
def Res_normal(name,F,drop,H,W):
    cl_input=Input(shape=(F,H,W))

    cl_conv1=conv_unit0(F,F,drop,H,W)(cl_input)

    cl_conv2=conv_unit0(F,F,drop,H,W)(cl_conv1)

    cl_out=Add()([cl_input,cl_conv2])

    cl_model=Model(inputs=cl_input,outputs=cl_out,name=name)

    return cl_model

def cpt_slice(x, h1, h2):
    return x[:,h1:h2,:,:]

# transfer Time vector to a number (e.g. corresponding to filters = 1 in Conv2D)
def T_trans(T,T_F,H,W):

    T_in=Input(shape=(T+7,H,W))
    T_mid=Conv2D(filters=T_F,kernel_size=(1,1),padding="same")(T_in)
    T_act=Activation('relu')(T_mid)
    T_fin=Conv2D(filters=1,kernel_size=(1,1),padding="same")(T_act)
    T_mul=Activation('relu')(T_fin)
    T_model=Model(inputs=T_in,outputs=T_mul)

    return T_model

# transfer Time vector and PoI matrix to time-weighted PoI matrix
def PT_trans(name,P_N,PT_F,T,T_F,H,W,isPT_F):
    if 1:
        poi_in=Input(shape=(P_N,H,W))
        # T_times/day + 7days/week
        time_in=Input(shape=(T+7,H,W))

        if P_N>=2:
            T_x0 =T_trans(T,T_F,H,W)(time_in)
            T_x1 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=3:
            T_x2 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=4:
            T_x3 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=5:
            T_x4 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=6:
            T_x5 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=7:
            T_x6 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=8:
            T_x7 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=9:
            T_x8 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=10:
            T_x9 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=11:
            T_x10=T_trans(T,T_F,H,W)(time_in)
        if P_N>=12:
            T_x11=T_trans(T,T_F,H,W)(time_in)
        if P_N>=13:
            T_x12=T_trans(T,T_F,H,W)(time_in)
        if P_N>=14:
            T_x13=T_trans(T,T_F,H,W)(time_in)
        if P_N>=15:
            T_x14=T_trans(T,T_F,H,W)(time_in)
        if P_N>=16:
            T_x15=T_trans(T,T_F,H,W)(time_in)


        if P_N==1:
            T_x=T_trans(T,T_F,H,W)(time_in)
        if P_N==2:
            T_x=Concatenate(axis=1)([T_x0,T_x1])
        if P_N==3:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2])
        if P_N==4:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3])
        if P_N==5:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4])
        if P_N==6:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5])
        if P_N==7:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6])
        if P_N==8:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7])
        if P_N==9:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8])
        if P_N==10:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9])
        if P_N==11:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10])
        if P_N==12:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11])
        if P_N==13:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12])
        if P_N==14:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12,T_x13])
        if P_N==15:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12,T_x13,T_x14])
        if P_N==16:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12,T_x13,T_x14,T_x15])

        poi_time=Multiply()([poi_in,T_x])
        if isPT_F:
            poi_time=Conv2D(filters=PT_F,kernel_size=(1,1),padding="same")(poi_time)
            print('PT_F = YES')
        else:
            print('PT_F = NO')
        PT_model=Model(inputs=[poi_in,time_in],outputs=poi_time,name=name)

        return PT_model

# main model
def DeepSTN(H=14,W=12,channel=1,      #H-map_height W-map_width channel-map_channel
            c=3,p=4,t=4,              #c-closeness p-period t-trend
            pre_F=64,conv_F=64,R_N=2, #pre_F-prepare_conv_featrue conv_F-resnet_conv_featrue R_N-resnet_number
            is_plus=True,             #use ResPlus or mornal convolution
            is_plus_efficient=False,  #use the efficient version of ResPlus
            plus=8,rate=2,            #rate-pooling_rate
            is_pt=False,               #use PoI and Time or not
            P_N=6,T_F=28,PT_F=6,T=24, #P_N-poi_number T_F-time_feature PT_F-poi_time_feature T-T_times/day
            drop=0,
            is_summary=False, #show detail
            lr=0.0001,
            kernel1=1, #kernel1 decides whether early-fusion uses conv_unit0 or conv_unit1, 1 recommended
            isPT_F=1): #isPT_F decides whether PT_model uses one more Conv after multiplying PoI and Time, 1 recommended

    all_channel = channel * (c+p+t)

    cut0 = int( 0 )
    cut1 = int( cut0 + channel*c )
    cut2 = int( cut1 + channel*p )
    cut3 = int( cut2 + channel*t )

    cpt_input=Input(shape=(all_channel,H,W))

    c_input=Lambda(cpt_slice,arguments={'h1':cut0,'h2':cut1})(cpt_input)
    p_input=Lambda(cpt_slice,arguments={'h1':cut1,'h2':cut2})(cpt_input)
    t_input=Lambda(cpt_slice,arguments={'h1':cut2,'h2':cut3})(cpt_input)

    c_out1=Conv2D(filters=pre_F,kernel_size=(1,1),padding="same")(c_input)
    p_out1=Conv2D(filters=pre_F,kernel_size=(1,1),padding="same")(p_input)
    t_out1=Conv2D(filters=pre_F,kernel_size=(1,1),padding="same")(t_input)

    if is_pt:
        poi_in=Input(shape=(P_N,H,W))
        # T_times/day + 7days/week
        time_in=Input(shape=(T+7,H,W))

        PT_model=PT_trans('PT_trans',P_N,PT_F,T,T_F,H,W,isPT_F)

        poi_time=PT_model([poi_in,time_in])

        cpt_con1=Concatenate(axis=1)([c_out1,p_out1,t_out1,poi_time]) # 그냥 CNN들이랑 더해서 Global PAttern 넣으려면 여기다 합치면 됨
        if kernel1:
            cpt=conv_unit1(pre_F*3+PT_F*isPT_F+P_N*(not isPT_F),conv_F,drop,H,W)(cpt_con1)
        else:
            cpt=conv_unit0(pre_F*3+PT_F*isPT_F+P_N*(not isPT_F),conv_F,drop,H,W)(cpt_con1)

    else:
        cpt_con1=Concatenate(axis=1)([c_out1,p_out1,t_out1]) # 그냥 CNN들이랑 더해서 Global PAttern 넣으려면 여기다 합치면 됨
        if kernel1:
            cpt=conv_unit1(pre_F*3,conv_F,drop,H,W)(cpt_con1)
        else:
            cpt=conv_unit0(pre_F*3,conv_F,drop,H,W)(cpt_con1)

    if is_plus:
        if is_plus_efficient:
            for i in range(R_N):
                cpt=Res_plus_E('Res_plus_'+str(i+1),conv_F,plus,rate,drop,H,W)(cpt)
        else:
            for i in range(R_N):
                cpt=Res_plus('Res_plus_'+str(i+1),conv_F,plus,rate,drop,H,W)(cpt)


    else:
        for i in range(R_N):
            cpt=Res_normal('Res_normal_'+str(i+1),conv_F,drop,H,W)(cpt)


    cpt_conv2=Activation('relu')(cpt)
    cpt_out2=BatchNormalization()(cpt_conv2)
    cpt_conv1=Dropout(drop)(cpt_out2)
    cpt_conv1=Conv2D(filters=channel,kernel_size=(1, 1),padding="same")(cpt_conv1)
    print('cpt_conv1 shape is', cpt_conv1.shape)
    # 여기서 GCN 결과랑 parametric based fusion, ST REsnet 코드 참조

    cpt_out1=Activation('tanh')(cpt_conv1)

    if is_pt:
        DeepSTN_model=Model(inputs=[cpt_input,poi_in,time_in],outputs=cpt_out1)
    else:
        DeepSTN_model=Model(inputs=cpt_input,outputs=cpt_out1)

    DeepSTN_model.compile(loss='mse', optimizer=Adam(lr), metrics=[rmse,mae])

    if is_summary:
        DeepSTN_model.summary()

    '''
    print('***** pre_F : ',pre_F       )
    print('***** conv_F: ',conv_F      )
    print('***** R_N   : ',R_N         )

    print('***** plus  : ',plus*is_plus)
    print('***** rate  : ',rate*is_plus)

    print('***** P_N   : ',P_N*is_pt   )
    print('***** T_F   : ',T_F*is_pt   )
    print('***** PT_F  : ',PT_F*is_pt*isPT_F )
    print('***** T     : ',T           )

    print('***** drop  : ',drop        )
    '''
    return DeepSTN_model

In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K


#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 16,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=3


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN




X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]

pre_F=64
conv_F=64
R_N=10

is_plus=True
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1

import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''


all_data shape:  (800, 1, 16, 12)
max= 60.0  min= 0.0
mean= -0.9908684895833333  variance= 0.04447277708632065
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000667 0.025101 0.006785]
Test  score: [0.00036  0.018962 0.005078]
RMSE  MAE
[[0.019  0.0051]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0063 0.0017]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000704 0.025801 0.006653]
Test  score: [0.000372 0.019288 0.004806]
RMSE  MAE
[[0.019  0.0051]
 [0.0193 0.0048]
 [0.     0.    ]]
RMSE  MAE
[0.0127 0.0033]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000699 0.02568  0.006696]
Test  score: [0.000369 0.019196 0.004858]
RMSE  MAE
[[0.019  0.0051]
 [0.0193 0.0048]
 [0.0192 0.0049]]
RMSE  MAE
[0.0191 0.0049]


In [ ]:
# m0_Ent

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m0_Ent.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 49.0  min= 0.0
mean= -0.9962797619047619  variance= 0.02770920949471694
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000296 0.016416 0.002599]
Test  score: [0.000202 0.014212 0.002054]
RMSE  MAE
[[0.0142 0.0021]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0047 0.0007]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000301 0.016542 0.002581]
Test  score: [0.000206 0.014336 0.00203 ]
RMSE  MAE
[[0.0142 0.0021]
 [0.0143 0.002 ]
 [0.     0.    ]]
RMSE  MAE
[0.0095 0.0014]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000363 0.018156 0.002777]
Test  score: [0.000241 0.015509 0.002153]
RMSE  MAE
[[0.0142 0.0021]
 [0.0143 0.002 ]
 [0.0155 0.0022]]
RMSE  MAE
[0.0147 0.0021]


In [ ]:
# m1_Col

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m1_Col.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 7.0  min= 0.0
mean= -0.9955078124999998  variance= 0.04561460128187354
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.001172 0.032246 0.003828]
Test  score: [0.000788 0.028068 0.002995]
RMSE  MAE
[[0.0281 0.003 ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0094 0.001 ]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.001202 0.032603 0.003604]
Test  score: [0.000819 0.028612 0.002769]
RMSE  MAE
[[0.0281 0.003 ]
 [0.0286 0.0028]
 [0.     0.    ]]
RMSE  MAE
[0.0189 0.0019]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.001172 0.032255 0.003812]
Test  score: [0.000797 0.028239 0.003005]
RMSE  MAE
[[0.0281 0.003 ]
 [0.0286 0.0028]
 [0.0282 0.003 ]]
RMSE  MAE
[0.0283 0.0029]


In [ ]:
# m2_mv

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m2_Ev.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 2.0  min= 0.0
mean= -0.9995963541666667  variance= 0.020408423180341476
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000303 0.010018 0.000357]
Test  score: [0.000232 0.015246 0.000289]
RMSE  MAE
[[0.0152 0.0003]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0051 0.0001]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000303 0.009992 0.000328]
Test  score: [0.000233 0.015248 0.00026 ]
RMSE  MAE
[[0.0152 0.0003]
 [0.0152 0.0003]
 [0.     0.    ]]
RMSE  MAE
[0.0102 0.0002]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000303 0.010064 0.000343]
Test  score: [0.000232 0.015243 0.000274]
RMSE  MAE
[[0.0152 0.0003]
 [0.0152 0.0003]
 [0.0152 0.0003]]
RMSE  MAE
[0.0152 0.0003]


In [ ]:
# m4_Night

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m4_Night.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 13.0  min= 0.0
mean= -0.9996183894230768  variance= 0.010054515338544382
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000116 0.007154 0.000336]
Test  score: [0.00004  0.006294 0.000244]
RMSE  MAE
[[0.0063 0.0002]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0021 0.0001]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000115 0.007132 0.000397]
Test  score: [0.000039 0.006254 0.000307]
RMSE  MAE
[[0.0063 0.0002]
 [0.0063 0.0003]
 [0.     0.    ]]
RMSE  MAE
[0.0042 0.0002]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000115 0.007135 0.000363]
Test  score: [0.000039 0.006253 0.000272]
RMSE  MAE
[[0.0063 0.0002]
 [0.0063 0.0003]
 [0.0063 0.0003]]
RMSE  MAE
[0.0063 0.0003]


In [ ]:
# m5_Outdoor

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m5_Outdoor.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 48.0  min= 0.0
mean= -0.9933797200520835  variance= 0.03452679972215169
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000646 0.022816 0.004677]
Test  score: [0.000224 0.01495  0.002947]
RMSE  MAE
[[0.015  0.0029]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.005 0.001]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000825 0.026212 0.005041]
Test  score: [0.000311 0.017637 0.003201]
RMSE  MAE
[[0.015  0.0029]
 [0.0176 0.0032]
 [0.     0.    ]]
RMSE  MAE
[0.0109 0.002 ]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000621 0.022309 0.004681]
Test  score: [0.000217 0.014746 0.002999]
RMSE  MAE
[[0.015  0.0029]
 [0.0176 0.0032]
 [0.0147 0.003 ]]
RMSE  MAE
[0.0158 0.003 ]


In [ ]:
# m6_Pro

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m6_Pro.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 68.0  min= 0.0
mean= -0.9916000306372551  variance= 0.04181876512912886
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.0009   0.027648 0.006297]
Test  score: [0.000406 0.020144 0.004604]
RMSE  MAE
[[0.0201 0.0046]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0067 0.0015]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000727 0.025054 0.006258]
Test  score: [0.000329 0.018128 0.004726]
RMSE  MAE
[[0.0201 0.0046]
 [0.0181 0.0047]
 [0.     0.    ]]
RMSE  MAE
[0.0128 0.0031]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000818 0.026523 0.006253]
Test  score: [0.000374 0.019336 0.004631]
RMSE  MAE
[[0.0201 0.0046]
 [0.0181 0.0047]
 [0.0193 0.0046]]
RMSE  MAE
[0.0192 0.0047]


In [ ]:
# m7_Res

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m7_Res.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 4.0  min= 0.0
mean= -0.997197265625  variance= 0.03973737211395832
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.001218 0.033212 0.002278]
Test  score: [0.000635 0.025195 0.001384]
RMSE  MAE
[[0.0252 0.0014]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0084 0.0005]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.00122  0.033247 0.002185]
Test  score: [0.000639 0.025282 0.001297]
RMSE  MAE
[[0.0252 0.0014]
 [0.0253 0.0013]
 [0.     0.    ]]
RMSE  MAE
[0.0168 0.0009]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.00122  0.033244 0.002211]
Test  score: [0.000639 0.025281 0.001323]
RMSE  MAE
[[0.0252 0.0014]
 [0.0253 0.0013]
 [0.0253 0.0013]]
RMSE  MAE
[0.0253 0.0013]


In [ ]:
# m3_Food

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m3_Food.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 98.0  min= 0.0
mean= -0.9913027476615647  variance= 0.054585593962263146
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.001029 0.029983 0.005875]
Test  score: [0.000488 0.022097 0.003977]
RMSE  MAE
[[0.0221 0.004 ]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0074 0.0013]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000773 0.026363 0.005634]
Test  score: [0.000389 0.019733 0.004034]
RMSE  MAE
[[0.0221 0.004 ]
 [0.0197 0.004 ]
 [0.     0.    ]]
RMSE  MAE
[0.0139 0.0027]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000811 0.026949 0.005769]
Test  score: [0.00039  0.019737 0.003987]
RMSE  MAE
[[0.0221 0.004 ]
 [0.0197 0.004 ]
 [0.0197 0.004 ]]
RMSE  MAE
[0.0205 0.004 ]


In [ ]:
# m9_Tra

import numpy as np

class MM:
    def __init__(self,MM_max,MM_min):
        self.max=MM_max
        self.min=MM_min

# 전처리에서 한 타임슬롯이 6시간으로 처리, T_period는 기존에 24여서 4로, T_trend는 기존에 일주일이어서 4*7로 수정
def lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness=1,T_period=4,T_trend=4*7):

    all_data=np.load('data3/m9_Tra.npy')
    all_data = all_data[:, np.newaxis, :, :] # 기존 코드와 dimension 맞춰주기 위해서 피쳐 축 하나 추가
    len_total,feature,map_height,map_width=all_data.shape
    #all_data=np.arange(48*24*7*256).reshape(-1,2,16,8)
    #len_total,feature,map_height,map_width=all_data.shape
    print('all_data shape: ',all_data.shape)
    #mm=MM(np.max(all_data),np.min(all_data))
    print('max=',np.max(all_data),' min=',np.min(all_data))

    #for time
    time=np.arange(len_total,dtype=int)
    #hour
    time_hour=time%T_period
    #matrix_hour=np.zeros([len_total,24,map_height,map_width]) # len_total(시간 단위) 마다 시간 별 맵 매트릭스 초기화
    matrix_hour=np.zeros([len_total,T_period,map_height,map_width]) # 24가 하루를 나타낸다고 보고, 한 슬랏에 6시간이니 24-> 4로 바꿈
    for i in range(len_total):
        matrix_hour[i,time_hour[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #day
    time_day=(time//T_period)%7
    matrix_day=np.zeros([len_total,7,map_height,map_width]) # len_total(시간 단위) 마다 요일 별 맵 매트릭스 초기화
    for i in range(len_total):
        matrix_day[i,time_day[i],:,:]=1 # 대각선 부분만 1로 초기화?
    #con
    matrix_T=np.concatenate((matrix_hour,matrix_day),axis=1) # 가로로 concat

    all_data=(2.0*all_data-(np.max(all_data)+np.min(all_data)))/(np.max(all_data)-np.min(all_data))
    print('mean=',np.mean(all_data),' variance=',np.std(all_data))

    if len_trend>0:
        number_of_skip_hours=T_trend*len_trend
    elif len_period>0:
        number_of_skip_hours=T_period*len_period
    elif len_closeness>0:
        number_of_skip_hours=T_closeness*len_closeness
    else:
        print("wrong")
    print('number_of_skip_hours:',number_of_skip_hours)

    Y=all_data[number_of_skip_hours:len_total]

    if len_closeness>0:
        X_closeness=all_data[number_of_skip_hours-T_closeness:len_total-T_closeness]
        for i in range(len_closeness-1):
            X_closeness=np.concatenate((X_closeness,all_data[number_of_skip_hours-T_closeness*(2+i):len_total-T_closeness*(2+i)]),axis=1)
    if len_period>0:
        X_period=all_data[number_of_skip_hours-T_period:len_total-T_period]
        for i in range(len_period-1):
            X_period=np.concatenate((X_period,all_data[number_of_skip_hours-T_period*(2+i):len_total-T_period*(2+i)]),axis=1)
    if len_trend>0:
        X_trend=all_data[number_of_skip_hours-T_trend:len_total-T_trend]
        for i in range(len_trend-1):
            X_trend=np.concatenate((X_trend,all_data[number_of_skip_hours-T_trend*(2+i):len_total-T_trend*(2+i)]),axis=1)

    matrix_T=matrix_T[number_of_skip_hours:]

    X_closeness_train=X_closeness[:-len_test]
    X_period_train=X_period[:-len_test]
    X_trend_train=X_trend[:-len_test]
    T_train=matrix_T[:-len_test]
    X_closeness_test=X_closeness[-len_test:]
    X_period_test=X_period[-len_test:]
    X_trend_test=X_trend[-len_test:]
    T_test=matrix_T[-len_test:]

    X_train=[X_closeness_train,X_period_train,X_trend_train]
    X_test=[X_closeness_test,X_period_test,X_trend_test]
    #X_train=np.concatenate((X_closeness_train,X_period_train,X_trend_train),axis=1)
    #X_test=np.concatenate((X_closeness_test,X_period_test,X_trend_test),axis=1)
    Y_train=Y[:-len_test]
    Y_test=Y[-len_test:]

    len_train=X_closeness_train.shape[0]
    len_test=X_closeness_test.shape[0]
    print('len_train='+str(len_train))
    print('len_test ='+str(len_test ))

    '''
    poi=np.load('DATA/dataBikeNYC/poi_data.npy')
    for i in range(poi.shape[0]):
        poi[i]=poi[i]/np.max(poi[i])
    P_train=np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_train,axis=0)
    P_test =np.repeat(poi.reshape(1,poi.shape[0],map_height,map_width),len_test ,axis=0)

    return X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,mm.max-mm.min
    '''
    return X_train,T_train,Y_train,X_test,T_test,Y_test,np.max(all_data)-np.min(all_data)


In [ ]:
from keras.callbacks import ModelCheckpoint
import numpy as np

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
import tensorflow as tf  #from V1707
from keras import backend as K

'''
#hyperparameters
epoch = 100  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0001  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 14,12,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 8*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=1


XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN
'''



X_train,T_train,Y_train,X_test,T_test,Y_test,MM = lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

index=np.arange(9)
#P_train=P_train[:,index,:,:]
#P_test =P_test [:,index,:,:]
'''
pre_F=64
conv_F=64
#R_N=2

#is_plus=False
print(R_N, is_plus)
plus=8
rate=1

is_pt=False
P_N=9
T_F=7*8
PT_F=9

drop=0.1
'''
import time
count=0
count_sum=iterate_num

iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

RMSE=np.zeros([iterate_num,1])
MAE =np.zeros([iterate_num,1])

for iterate_index in range(iterate_num):
    count=count+1
    time_start=time.time()
    iterate=iterate_loop[iterate_index]

    #print("***** conv_model *****")
    model=DeepSTN(H=H,W=W,channel=channel,
                  c=len_closeness,p=len_period,
                  pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                  is_plus=is_plus,
                  plus=plus,rate=rate,
                  is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                  drop=drop)

    #file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
    #train conv_model

    if train00:
        '''
        model_checkpoint=ModelCheckpoint(
                filepath=file_conv,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=True,
                mode='min',
                period=1
            )
        '''
        #print('=' * 10)
        #print("***** training conv_model *****")
        history = model.fit(X_train, Y_train,
                            epochs=epoch,
                            batch_size=batch_size,
                            validation_split=0.1,
                            #callbacks=[model_checkpoint],
                            verbose=0,
                            #validation_data=(X_test, Y_test)
                            )

    #print('=' * 10)
    #print('***** evaluate *****')
    #model.load_weights(file_conv)

    score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
    print('              mse     rmse    mae')
    print('Train score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))
    score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
    print('Test  score:',end=' ')
    np.set_printoptions(precision=6, suppress=True)
    print(np.array(score))


    RMSE[iterate_index,0]=score[1]
    MAE [iterate_index,0]=score[2]

    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

    np.set_printoptions(precision=4, suppress=True)
    print('RMSE  MAE')
    print(for_show)

    for_show=np.mean(for_show,axis=0)
    print('RMSE  MAE')
    print(for_show)

    #np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])
    '''
    time_end=time.time()
    print('iterate cost',time_end-time_start)
    print(str(count)+'/'+str(count_sum))
    '''

all_data shape:  (800, 1, 16, 12)
max= 37.0  min= 0.0
mean= -0.9933016610360359  variance= 0.037008620307463456
number_of_skip_hours: 224
len_train=464
len_test =112
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000525 0.021563 0.004615]
Test  score: [0.000277 0.016644 0.003395]
RMSE  MAE
[[0.0166 0.0034]
 [0.     0.    ]
 [0.     0.    ]]
RMSE  MAE
[0.0055 0.0011]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000533 0.021737 0.004544]
Test  score: [0.000277 0.016657 0.003304]
RMSE  MAE
[[0.0166 0.0034]
 [0.0167 0.0033]
 [0.     0.    ]]
RMSE  MAE
[0.0111 0.0022]
cpt_conv1 shape is (None, 1, 16, 12)
              mse     rmse    mae
Train score: [0.000514 0.021345 0.004889]
Test  score: [0.00027  0.016445 0.003718]
RMSE  MAE
[[0.0166 0.0034]
 [0.0167 0.0033]
 [0.0164 0.0037]]
RMSE  MAE
[0.0166 0.0035]


In [ ]:
# "DeepSTN_net.py"

from keras import backend as K
K.set_image_data_format('channels_first')
import numpy as np

from keras.optimizers import Adam
from keras.layers import Input,Activation,Dropout,BatchNormalization,AveragePooling2D,ZeroPadding2D,Multiply
from keras.layers import Lambda,Reshape,Concatenate,Add,Permute,TimeDistributed
from keras.models import Model
from keras.layers.convolutional import Conv2D
import PPT3_network.metrics as metrics

# Relu-BN-Conv2D 3x3
def conv_unit0(Fin,Fout,drop,H,W):
    unit_input=Input(shape=(Fin,H,W))

    unit_conv=Activation('relu')(unit_input)
    unit_conv=BatchNormalization()(unit_conv)
    unit_conv=Dropout(drop)(unit_conv)
    unit_output=Conv2D(filters=Fout,kernel_size=(3,3),padding="same")(unit_conv)
    unit_model=Model(inputs=unit_input,outputs=unit_output)
    print('kernel=(3,3)')
    return unit_model

# Relu-BN-Conv2D 1x1
def conv_unit1(Fin,Fout,drop,H,W):
    unit_input=Input(shape=(Fin,H,W))

    unit_conv=Activation('relu')(unit_input)
    unit_conv=BatchNormalization()(unit_conv)
    unit_conv=Dropout(drop)(unit_conv)
    unit_output=Conv2D(filters=Fout,kernel_size=(1,1),padding="same")(unit_conv)
    unit_model=Model(inputs=unit_input,outputs=unit_output)
    print('kernel=(1,1)')
    return unit_model

# efficient version of Res_plus
def Res_plus_E(name,F,Fplus,rate,drop,H,W):
    cl_input=Input(shape=(F,H,W))

    #normal channels
    cl_conv1A=conv_unit0(F,F-Fplus,drop,H,W)(cl_input)

    #separated channels
    if rate == 1:
        cl_conv1B=cl_input
    if rate !=1:
        cl_conv1B=AveragePooling2D(pool_size=(rate,rate),strides=(rate,rate),padding="valid")(cl_input)

    HR,WR=int(np.floor(H/rate)),int(np.floor(W/rate))
    cl_conv1B=Activation('relu')(cl_conv1B)
    cl_conv1B=BatchNormalization()(cl_conv1B)

    cl_conv1B=Conv2D(filters=Fplus,kernel_size=(1,1),use_bias=False,padding="same")(cl_conv1B)
    cl_conv1B=Reshape((Fplus,1,HR,WR),input_shape=(Fplus,HR,WR))(cl_conv1B)
    attention=Conv2D(filters=H*W,kernel_size=(HR,WR),use_bias=False,W_constraint=nonneg(),padding="valid")
    cl_conv1B=TimeDistributed(attention)(cl_conv1B)
    cl_conv1B=Reshape((Fplus,H,W),input_shape=(Fplus,H*W,1,1))(cl_conv1B)

    #merge
    cl_conv1=Concatenate(axis=1)([cl_conv1A,cl_conv1B])

    cl_conv2=conv_unit0(F,F,drop,H,W)(cl_conv1)

    cl_out=Add()([cl_input,cl_conv2])

    cl_model=Model(inputs=cl_input,outputs=cl_out,name=name)

    return cl_model

# new resdual block
def Res_plus(name,F,Fplus,rate,drop,H,W):
    cl_input=Input(shape=(F,H,W))

    cl_conv1A=conv_unit0(F,F-Fplus,drop,H,W)(cl_input)

    if rate == 1:
        cl_conv1B=cl_input
    if rate !=1:
        cl_conv1B=AveragePooling2D(pool_size=(rate,rate),strides=(rate,rate),padding="valid")(cl_input)

    cl_conv1B=Activation('relu')(cl_conv1B)
    cl_conv1B=BatchNormalization()(cl_conv1B)

    plus_conv=Conv2D(filters=Fplus*H*W,kernel_size=(int(np.floor(H/rate)),int(np.floor(W/rate))),padding="valid")

    cl_conv1B=plus_conv(cl_conv1B)

    cl_conv1B=Reshape((Fplus,H,W))(cl_conv1B)

    cl_conv1=Concatenate(axis=1)([cl_conv1A,cl_conv1B])

    cl_conv2=conv_unit0(F,F,drop,H,W)(cl_conv1)

    cl_out=Add()([cl_input,cl_conv2])

    cl_model=Model(inputs=cl_input,outputs=cl_out,name=name)

    return cl_model

# normal residual block
def Res_normal(name,F,drop,H,W):
    cl_input=Input(shape=(F,H,W))

    cl_conv1=conv_unit0(F,F,drop,H,W)(cl_input)

    cl_conv2=conv_unit0(F,F,drop,H,W)(cl_conv1)

    cl_out=Add()([cl_input,cl_conv2])

    cl_model=Model(inputs=cl_input,outputs=cl_out,name=name)

    return cl_model

def cpt_slice(x, h1, h2):
    return x[:,h1:h2,:,:]

# transfer Time vector to a number (e.g. corresponding to filters = 1 in Conv2D)
def T_trans(T,T_F,H,W):

    T_in=Input(shape=(T+7,H,W))
    T_mid=Conv2D(filters=T_F,kernel_size=(1,1),padding="same")(T_in)
    T_act=Activation('relu')(T_mid)
    T_fin=Conv2D(filters=1,kernel_size=(1,1),padding="same")(T_act)
    T_mul=Activation('relu')(T_fin)
    T_model=Model(inputs=T_in,outputs=T_mul)

    return T_model

# transfer Time vector and PoI matrix to time-weighted PoI matrix
def PT_trans(name,P_N,PT_F,T,T_F,H,W,isPT_F):
    if 1:
        poi_in=Input(shape=(P_N,H,W))
        # T_times/day + 7days/week
        time_in=Input(shape=(T+7,H,W))

        if P_N>=2:
            T_x0 =T_trans(T,T_F,H,W)(time_in)
            T_x1 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=3:
            T_x2 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=4:
            T_x3 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=5:
            T_x4 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=6:
            T_x5 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=7:
            T_x6 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=8:
            T_x7 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=9:
            T_x8 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=10:
            T_x9 =T_trans(T,T_F,H,W)(time_in)
        if P_N>=11:
            T_x10=T_trans(T,T_F,H,W)(time_in)
        if P_N>=12:
            T_x11=T_trans(T,T_F,H,W)(time_in)
        if P_N>=13:
            T_x12=T_trans(T,T_F,H,W)(time_in)
        if P_N>=14:
            T_x13=T_trans(T,T_F,H,W)(time_in)
        if P_N>=15:
            T_x14=T_trans(T,T_F,H,W)(time_in)
        if P_N>=16:
            T_x15=T_trans(T,T_F,H,W)(time_in)


        if P_N==1:
            T_x=T_trans(T,T_F,H,W)(time_in)
        if P_N==2:
            T_x=Concatenate(axis=1)([T_x0,T_x1])
        if P_N==3:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2])
        if P_N==4:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3])
        if P_N==5:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4])
        if P_N==6:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5])
        if P_N==7:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6])
        if P_N==8:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7])
        if P_N==9:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8])
        if P_N==10:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9])
        if P_N==11:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10])
        if P_N==12:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11])
        if P_N==13:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12])
        if P_N==14:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12,T_x13])
        if P_N==15:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12,T_x13,T_x14])
        if P_N==16:
            T_x=Concatenate(axis=1)([T_x0,T_x1,T_x2,T_x3,T_x4,T_x5,T_x6,T_x7,T_x8,T_x9,T_x10,T_x11,T_x12,T_x13,T_x14,T_x15])

        poi_time=Multiply()([poi_in,T_x])
        if isPT_F:
            poi_time=Conv2D(filters=PT_F,kernel_size=(1,1),padding="same")(poi_time)
            print('PT_F = YES')
        else:
            print('PT_F = NO')
        PT_model=Model(inputs=[poi_in,time_in],outputs=poi_time,name=name)

        return PT_model

# main model
def DeepSTN(H=12,W=14,channel=1,      #H-map_height W-map_width channel-map_channel
            c=3,p=4,t=4,              #c-closeness p-period t-trend
            pre_F=64,conv_F=64,R_N=2, #pre_F-prepare_conv_featrue conv_F-resnet_conv_featrue R_N-resnet_number
            is_plus=True,             #use ResPlus or mornal convolution
            is_plus_efficient=False,  #use the efficient version of ResPlus
            plus=8,rate=2,            #rate-pooling_rate
            is_pt=True,               #use PoI and Time or not
            P_N=6,T_F=28,PT_F=6,T=24, #P_N-poi_number T_F-time_feature PT_F-poi_time_feature T-T_times/day
            drop=0,
            is_summary=True, #show detail
            lr=0.0002,
            kernel1=1, #kernel1 decides whether early-fusion uses conv_unit0 or conv_unit1, 1 recommended
            isPT_F=1): #isPT_F decides whether PT_model uses one more Conv after multiplying PoI and Time, 1 recommended

    all_channel = channel * (c+p+t)

    cut0 = int( 0 )
    cut1 = int( cut0 + channel*c )
    cut2 = int( cut1 + channel*p )
    cut3 = int( cut2 + channel*t )

    cpt_input=Input(shape=(all_channel,H,W))

    c_input=Lambda(cpt_slice,arguments={'h1':cut0,'h2':cut1})(cpt_input)
    p_input=Lambda(cpt_slice,arguments={'h1':cut1,'h2':cut2})(cpt_input)
    t_input=Lambda(cpt_slice,arguments={'h1':cut2,'h2':cut3})(cpt_input)

    c_out1=Conv2D(filters=pre_F,kernel_size=(1,1),padding="same")(c_input)
    p_out1=Conv2D(filters=pre_F,kernel_size=(1,1),padding="same")(p_input)
    t_out1=Conv2D(filters=pre_F,kernel_size=(1,1),padding="same")(t_input)

    if is_pt:
        poi_in=Input(shape=(P_N,H,W))
        # T_times/day + 7days/week
        time_in=Input(shape=(T+7,H,W))

        PT_model=PT_trans('PT_trans',P_N,PT_F,T,T_F,H,W,isPT_F)

        poi_time=PT_model([poi_in,time_in])

        cpt_con1=Concatenate(axis=1)([c_out1,p_out1,t_out1,poi_time]) # 그냥 CNN들이랑 더해서 Global PAttern 넣으려면 여기다 합치면 됨
        if kernel1:
            cpt=conv_unit1(pre_F*3+PT_F*isPT_F+P_N*(not isPT_F),conv_F,drop,H,W)(cpt_con1)
        else:
            cpt=conv_unit0(pre_F*3+PT_F*isPT_F+P_N*(not isPT_F),conv_F,drop,H,W)(cpt_con1)

    else:
        cpt_con1=Concatenate(axis=1)([c_out1,p_out1,t_out1]) # 그냥 CNN들이랑 더해서 Global PAttern 넣으려면 여기다 합치면 됨
        if kernel1:
            cpt=conv_unit1(pre_F*3,conv_F,drop,H,W)(cpt_con1)
        else:
            cpt=conv_unit0(pre_F*3,conv_F,drop,H,W)(cpt_con1)

    if is_plus:
        if is_plus_efficient:
            for i in range(R_N):
                cpt=Res_plus_E('Res_plus_'+str(i+1),conv_F,plus,rate,drop,H,W)(cpt)
        else:
            for i in range(R_N):
                cpt=Res_plus('Res_plus_'+str(i+1),conv_F,plus,rate,drop,H,W)(cpt)

    else:
        for i in range(R_N):
            cpt=Res_normal('Res_normal_'+str(i+1),conv_F,drop,H,W)(cpt)


    cpt_conv2=Activation('relu')(cpt)
    cpt_out2=BatchNormalization()(cpt_conv2)
    cpt_conv1=Dropout(drop)(cpt_out2)
    cpt_conv1=Conv2D(filters=channel,kernel_size=(1, 1),padding="same")(cpt_conv1)

    # 여기서 GCN 결과랑 parametric based fusion, ST REsnet 코드 참조

    cpt_out1=Activation('tanh')(cpt_conv1)

    if is_pt:
        DeepSTN_model=Model(inputs=[cpt_input,poi_in,time_in],outputs=cpt_out1)
    else:
        DeepSTN_model=Model(inputs=cpt_input,outputs=cpt_out1)

    DeepSTN_model.compile(loss='mse', optimizer=Adam(lr), metrics=[metrics.rmse,metrics.mae])

    if is_summary:
        DeepSTN_model.summary()


    print('***** pre_F : ',pre_F       )
    print('***** conv_F: ',conv_F      )
    print('***** R_N   : ',R_N         )

    print('***** plus  : ',plus*is_plus)
    print('***** rate  : ',rate*is_plus)

    print('***** P_N   : ',P_N*is_pt   )
    print('***** T_F   : ',T_F*is_pt   )
    print('***** PT_F  : ',PT_F*is_pt*isPT_F )
    print('***** T     : ',T           )

    print('***** drop  : ',drop        )

    return DeepSTN_model

In [ ]:
# "metrics.py"

# import numpy as np
from keras import backend as K

def mean_squared_error(y_true, y_pred):
    return K.mean(K.square(y_pred - y_true))


def root_mean_square_error(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5

def rmse(y_true, y_pred):
    return mean_squared_error(y_true, y_pred) ** 0.5


# aliases
mse = MSE = mean_squared_error
# rmse = RMSE = root_mean_square_error


def masked_mean_squared_error(y_true, y_pred):
    idx = (y_true > 1e-6).nonzero()
    return K.mean(K.square(y_pred[idx] - y_true[idx]))

def masked_rmse(y_true, y_pred):
    return masked_mean_squared_error(y_true, y_pred) ** 0.5


def mean_absolute_error(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))

def mae(y_true, y_pred):
    return K.mean(K.abs(y_pred - y_true))


threshold=0.05

def mean_absolute_percentage_error(y_true, y_pred):
    return K.mean( K.abs(y_pred-y_true) / K.maximum(K.cast(threshold,'float32'),y_true+1.0) )

def mape(y_true, y_pred):
    return K.mean( K.abs(y_pred-y_true) / K.maximum(K.cast(threshold,'float32'),y_true+1.0) )



In [ ]:
#ComparisionBikeNYC.py


#from __future__ import print_function
#from DATA.lzq_read_data_time_poi import lzq_load_data
from keras.callbacks import ModelCheckpoint
#import cPickle as pickle
import numpy as np
#import math

NO=4
#for reproduction
seed=1
for i in range(NO):
    seed=seed*10+7
seed=seed*10+7
np.random.seed(seed)
#from ipdb import set_trace
#set_trace()

#for GPU in Lab
device=6

import os
os.environ["CUDA_VISIBLE_DEVICES"]=str(device)
import tensorflow as tf  #from V1707
config=tf.ConfigProto()  #from V1707
config.gpu_options.allow_growth=True  #from V1707
#config.gpu_options.per_process_gpu_memory_fraction=0.5
sess=tf.Session(config=config)  #from V1707
#import keras.backend.tensorflow_backend as KTF
#KTF._set_session(tf.Session(config=config))
import setproctitle  #from V1707
setproctitle.setproctitle('Comprison Start! @ ZiqianLin')  #from V1707

from keras import backend as K
K.set_image_data_format('channels_first')


#hyperparameters
epoch = 1#350  # number of epoch at training stage
batch_size = 32  # batch size
lr = 0.0002  # learning rate

#H,W,channel = 21,12,2   # grid size
H,W,channel = 18,15,1   # grid size

#T = 24*1  # number of time intervals in one day
T = 4*1  # number of time intervals in one day

len_closeness = 3  # length of closeness dependent sequence
len_period = 4  # length of peroid dependent sequence
len_trend = 4  # length of trend dependent sequence

T_closeness,T_period,T_trend=1,T,T*7

# last 7 days for testing data
days_test = 14
len_test = T * days_test

#the number of repetition and if retrain the model
iterate_num=10

XDST=0  #DST
X11=1   #DSTN+ResPlus+PoI&Time
X10=1   #DSTN+ResPlus
X01=0   #DSTN+PoI&Time
X00=0   #DSTN

trainDST=1  #DST
train11=1   #DSTN+ResPlus+PoI&Time
train10=1   #DSTN+ResPlus
train01=1   #DSTN+PoI&Time
train00=1   #DSTN



#DST result
if XDST:
    setproctitle.setproctitle('BJMobile DST @ ZiqianLin')  #from V1707

    print("loading data...")
    X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

    R_N = 4   # number of residual units

    from keras.optimizers import Adam
    from DST_network.STResNet import stresnet
    import DST_network.metrics as metrics

    def build_model(external_dim,CFN):
        c_conf = (len_closeness, channel, H, W) if len_closeness > 0 else None
        p_conf = (len_period,    channel, H, W) if len_period    > 0 else None
        t_conf = (len_trend,     channel, H, W) if len_trend     > 0 else None

        model = stresnet(c_conf=c_conf, p_conf=p_conf, t_conf=t_conf,
                         external_dim=external_dim, nb_residual_unit=R_N, CF=CFN)

        adam = Adam(lr=lr)
        model.compile(loss='mse', optimizer=adam, metrics=[metrics.rmse,metrics.mae])
        model.summary()
        #from keras.utils.visualize_util import plot
        #plot(model, to_file='model.png', show_shapes=True)
        return model


    CF=64

    iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

    RMSE=np.zeros([iterate_num,1])
    MAE =np.zeros([iterate_num,1])
    count_sum=iterate_num

    import time

    count=0


    for iterate_index in range(iterate_num):
        count=count+1
        iterate=iterate_loop[iterate_index]

        time_start=time.time()

        F='DST_MODEL/dst_model_'+str(iterate)+'_.hdf5'

        model = build_model(external_dim=False,CFN=CF)
        if trainDST:
            model_checkpoint=ModelCheckpoint(
                filepath=F,
                monitor='val_rmse',
                verbose=1,
                save_best_only=True,
                save_weights_only=False,
                mode='min',
                period=1)

            print('=' * 10)
            print("training model...")
            history = model.fit(X_train, Y_train,
                                nb_epoch=epoch,
                                batch_size=batch_size,
                                validation_split=0.1,
                                callbacks=[model_checkpoint],
                                verbose=1)

        print('=' * 10)
        print('evaluating using the model that has the best loss on the valid set')
        model.load_weights(F)

        score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
        print('              mse     rmse    mae')
        print('Train score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        score = model.evaluate(X_test, Y_test, batch_size=Y_test.shape[0], verbose=0)
        print('Test  score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))

        RMSE[iterate_index,0]=score[1]
        MAE [iterate_index,0]=score[2]

        for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

        np.set_printoptions(precision=4, suppress=True)
        print('RMSE  MAE')
        print(for_show)

        for_show=np.mean(for_show,axis=0)
        print('RMSE  MAE')
        print(for_show)

        np.save('DST_SCORE/dst_score.npy',[RMSE,MAE])

        time_end=time.time()

        print('totally cost',time_end-time_start)
        print(str(count)+'/'+str(count_sum))



#DSTN+ResPlus+PoI&Time
if X11:
    setproctitle.setproctitle('BJMobile DSTN+ResPlus+PoI&Time @ ZiqianLin')  #from V1707
    from DeepSTN_network.DeepSTN_net import DeepSTN

    X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

    X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
    X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

    index=np.arange(9)
    P_train=P_train[:,index,:,:]
    P_test =P_test [:,index,:,:]

    pre_F=64
    conv_F=64
    R_N=2

    is_plus=True
    plus=8
    rate=1

    is_pt=True
    P_N=9
    T_F=7*8
    PT_F=9

    drop=0.1

    import time
    count=0
    count_sum=iterate_num

    iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

    RMSE=np.zeros([iterate_num,1])
    MAE =np.zeros([iterate_num,1])

    for iterate_index in range(iterate_num):
        count=count+1
        time_start=time.time()
        iterate=iterate_loop[iterate_index]

        print("***** conv_model *****")
        model=DeepSTN(H=H,W=W,channel=channel,
                      c=len_closeness,p=len_period,
                      pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                      is_plus=is_plus,
                      plus=plus,rate=rate,
                      is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                      drop=drop)

        file_conv='DeepSTN_11/MODEL/DeepSTN_11_model_'+str(iterate)+'.hdf5'
        #train conv_model
        if train11:
            model_checkpoint=ModelCheckpoint(
                    filepath=file_conv,
                    monitor='val_rmse',
                    verbose=1,
                    save_best_only=True,
                    save_weights_only=True,
                    mode='min',
                    period=1
                )

            print('=' * 10)
            print("***** training conv_model *****")
            history = model.fit([X_train,P_train,T_train], Y_train,
                                epochs=epoch,
                                batch_size=batch_size,
                                validation_split=0.1,
                                callbacks=[model_checkpoint],
                                verbose=1)

        print('=' * 10)
        print('***** evaluate *****')
        model.load_weights(file_conv)

        score = model.evaluate([X_train,P_train,T_train], Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
        print('              mse     rmse    mae')
        print('Train score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))
        score = model.evaluate([X_test ,P_test ,T_test ], Y_test, batch_size=Y_test.shape[0], verbose=0)
        print('Test  score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))

        RMSE[iterate_index,0]=score[1]
        MAE [iterate_index,0]=score[2]

        for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

        np.set_printoptions(precision=4, suppress=True)
        print('RMSE  MAE')
        print(for_show)

        for_show=np.mean(for_show,axis=0)
        print('RMSE  MAE')
        print(for_show)

        np.save('DeepSTN_11/SCORE/DeepSTN_11_score3.npy',[RMSE,MAE])

        time_end=time.time()
        print('iterate cost',time_end-time_start)
        print(str(count)+'/'+str(count_sum))



#DSTN+ResPlus
if X10:
    setproctitle.setproctitle('BJMobile DSTN+ResPlus @ ZiqianLin')  #from V1707
    from DeepSTN_network.DeepSTN_net import DeepSTN

    X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

    X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
    X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

    #index=np.arange(9)
    #P_train=P_train[:,index,:,:]
    #P_test =P_test [:,index,:,:]

    pre_F=64
    conv_F=64
    R_N=2

    is_plus=True
    plus=8
    rate=1

    is_pt=False
    P_N=9
    T_F=7*8
    PT_F=9

    drop=0.1

    import time
    count=0
    count_sum=iterate_num

    iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

    RMSE=np.zeros([iterate_num,1])
    MAE =np.zeros([iterate_num,1])

    for iterate_index in range(iterate_num):
        count=count+1
        time_start=time.time()
        iterate=iterate_loop[iterate_index]

        print("***** conv_model *****")
        model=DeepSTN(H=H,W=W,channel=channel,
                      c=len_closeness,p=len_period,
                      pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                      is_plus=is_plus,
                      plus=plus,rate=rate,
                      is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                      drop=drop)

        file_conv='DeepSTN_10/MODEL/DeepSTN_10_model_'+str(iterate)+'.hdf5'
        #train conv_model
        if train10:
            model_checkpoint=ModelCheckpoint(
                    filepath=file_conv,
                    monitor='val_rmse',
                    verbose=1,
                    save_best_only=True,
                    save_weights_only=True,
                    mode='min',
                    period=1
                )

            print('=' * 10)
            print("***** training conv_model *****")
            history = model.fit(X_train, Y_train,
                                epochs=epoch,
                                batch_size=batch_size,
                                validation_split=0.1,
                                callbacks=[model_checkpoint],
                                verbose=1)

        print('=' * 10)
        print('***** evaluate *****')
        model.load_weights(file_conv)

        score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
        print('              mse     rmse    mae')
        print('Train score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))
        score = model.evaluate(X_test,  Y_test, batch_size=Y_test.shape[0], verbose=0)
        print('Test  score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))

        RMSE[iterate_index,0]=score[1]
        MAE [iterate_index,0]=score[2]

        for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

        np.set_printoptions(precision=4, suppress=True)
        print('RMSE  MAE')
        print(for_show)

        for_show=np.mean(for_show,axis=0)
        print('RMSE  MAE')
        print(for_show)

        np.save('DeepSTN_10/SCORE/DeepSTN_10_score.npy',[RMSE,MAE])

        time_end=time.time()
        print('iterate cost',time_end-time_start)
        print(str(count)+'/'+str(count_sum))



#DSTN+PoI&Time
if X01:
    setproctitle.setproctitle('BJMobile DSTN+PoI&Time @ ZiqianLin')  #from V1707
    from DeepSTN_network.DeepSTN_net import DeepSTN

    X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

    X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
    X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

    #index=np.arange(9)
    #P_train=P_train[:,index,:,:]
    #P_test =P_test [:,index,:,:]

    pre_F=64
    conv_F=64
    R_N=2

    is_plus=False
    plus=8
    rate=1

    is_pt=True
    P_N=9
    T_F=7*8
    PT_F=9

    drop=0.1

    import time
    count=0
    count_sum=iterate_num

    iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

    RMSE=np.zeros([iterate_num,1])
    MAE =np.zeros([iterate_num,1])

    for iterate_index in range(iterate_num):
        count=count+1
        time_start=time.time()
        iterate=iterate_loop[iterate_index]

        print("***** conv_model *****")
        model=DeepSTN(H=H,W=W,channel=channel,
                      c=len_closeness,p=len_period,
                      pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                      is_plus=is_plus,
                      plus=plus,rate=rate,
                      is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                      drop=drop)

        file_conv='DeepSTN_01/MODEL/DeepSTN_01_model_'+str(iterate)+'.hdf5'
        #train conv_model
        if train01:
            model_checkpoint=ModelCheckpoint(
                    filepath=file_conv,
                    monitor='val_rmse',
                    verbose=1,
                    save_best_only=True,
                    save_weights_only=True,
                    mode='min',
                    period=1
                )

            print('=' * 10)
            print("***** training conv_model *****")
            history = model.fit([X_train,P_train,T_train], Y_train,
                                epochs=epoch,
                                batch_size=batch_size,
                                validation_split=0.1,
                                callbacks=[model_checkpoint],
                                verbose=1)

        print('=' * 10)
        print('***** evaluate *****')
        model.load_weights(file_conv)

        score = model.evaluate([X_train,P_train,T_train], Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
        print('              mse     rmse    mae')
        print('Train score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))
        score = model.evaluate([X_test, P_test, T_test ],  Y_test,  batch_size=Y_test.shape[0], verbose=0)
        print('Test  score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))

        RMSE[iterate_index,0]=score[1]
        MAE [iterate_index,0]=score[2]

        for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

        np.set_printoptions(precision=4, suppress=True)
        print('RMSE  MAE')
        print(for_show)

        for_show=np.mean(for_show,axis=0)
        print('RMSE  MAE')
        print(for_show)

        np.save('DeepSTN_01/SCORE/DeepSTN_01_score.npy',[RMSE,MAE])

        time_end=time.time()
        print('iterate cost',time_end-time_start)
        print(str(count)+'/'+str(count_sum))



#DSTN
if X00:
    setproctitle.setproctitle('BJMobile DSTN+PoI&Time @ ZiqianLin')  #from V1707
    from DeepSTN_network.DeepSTN_net import DeepSTN

    X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

    X_train=np.concatenate((X_train[0],X_train[1],X_train[2]),axis=1)
    X_test =np.concatenate((X_test[0], X_test[1], X_test[2] ),axis=1)

    index=np.arange(9)
    P_train=P_train[:,index,:,:]
    P_test =P_test [:,index,:,:]

    pre_F=64
    conv_F=64
    R_N=2

    is_plus=False
    plus=8
    rate=1

    is_pt=False
    P_N=9
    T_F=7*8
    PT_F=9

    drop=0.1

    import time
    count=0
    count_sum=iterate_num

    iterate_loop=np.arange(iterate_num)+1+iterate_num*(NO-1)

    RMSE=np.zeros([iterate_num,1])
    MAE =np.zeros([iterate_num,1])

    for iterate_index in range(iterate_num):
        count=count+1
        time_start=time.time()
        iterate=iterate_loop[iterate_index]

        print("***** conv_model *****")
        model=DeepSTN(H=H,W=W,channel=channel,
                      c=len_closeness,p=len_period,
                      pre_F=pre_F,conv_F=conv_F,R_N=R_N,
                      is_plus=is_plus,
                      plus=plus,rate=rate,
                      is_pt=is_pt,P_N=P_N,T_F=T_F,PT_F=PT_F,T=T,
                      drop=drop)

        file_conv='DeepSTN_00/MODEL/DeepSTN_00_model_'+str(iterate)+'.hdf5'
        #train conv_model
        if train00:
            model_checkpoint=ModelCheckpoint(
                    filepath=file_conv,
                    monitor='val_rmse',
                    verbose=1,
                    save_best_only=True,
                    save_weights_only=True,
                    mode='min',
                    period=1
                )

            print('=' * 10)
            print("***** training conv_model *****")
            history = model.fit(X_train, Y_train,
                                epochs=epoch,
                                batch_size=batch_size,
                                validation_split=0.1,
                                callbacks=[model_checkpoint],
                                verbose=1)

        print('=' * 10)
        print('***** evaluate *****')
        model.load_weights(file_conv)

        score = model.evaluate(X_train, Y_train, batch_size=Y_train.shape[0] // 48, verbose=0)
        print('              mse     rmse    mae')
        print('Train score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))
        score = model.evaluate(X_test,  Y_test,  batch_size=Y_test.shape[0], verbose=0)
        print('Test  score:',end=' ')
        np.set_printoptions(precision=6, suppress=True)
        print(np.array(score))

        RMSE[iterate_index,0]=score[1]
        MAE [iterate_index,0]=score[2]

        for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2

        np.set_printoptions(precision=4, suppress=True)
        print('RMSE  MAE')
        print(for_show)

        for_show=np.mean(for_show,axis=0)
        print('RMSE  MAE')
        print(for_show)

        np.save('DeepSTN_00/SCORE/DeepSTN_00_score.npy',[RMSE,MAE])

        time_end=time.time()
        print('iterate cost',time_end-time_start)
        print(str(count)+'/'+str(count_sum))



#Comparison
X_train,T_train,P_train,Y_train,X_test,T_test,P_test,Y_test,MM=lzq_load_data(len_test,len_closeness,len_period,len_trend,T_closeness,T_period,T_trend)

np.set_printoptions(precision=4, suppress=True)
print('MODEL                     RMSE  MAE')
if 0:
    print('ResNet                  :',end=' ')
    [RMSE,MAE]=np.load('DST_SCORE/dst_score.npy')
    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2
    print(for_show)
if 0:
    print('DeepSTN                 :',end=' ')
    [RMSE,MAE]=np.load('DeepSTN_00/SCORE/DeepSTN_00_score.npy')
    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2
    for_show=np.mean(for_show,axis=0)
    print(for_show)
if 1:
    print('DeepSTN+ResPlus         :',end=' ')
    [RMSE,MAE]=np.load('DeepSTN_10/SCORE/DeepSTN_10_score.npy')
    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2
    for_show=np.mean(for_show,axis=0)
    print(for_show)
if 0:
    print('DeepSTN+PoI&Time        :',end=' ')
    [RMSE,MAE]=np.load('DeepSTN_01/SCORE/DeepSTN_01_score.npy')
    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2
    for_show=np.mean(for_show,axis=0)
    print(for_show)
if 1:
    print('DeepSTN+ResPlus+PoI&Time:',end=' ')
    [RMSE,MAE]=np.load('DeepSTN_11/SCORE/DeepSTN_11_score3.npy')
    for_show=np.concatenate([RMSE,MAE],axis=1)*MM/2
    for_show=np.mean(for_show,axis=0)
    print(for_show)
